# Lemmatization and full text search (FTS)

The task is concentrated on using full text search engine (ElasticSearch) to perform basic search operations in a text corpus.

### Tasks

1. Install ElasticSearch (ES).
2. Install an ES plugin for Polish https://github.com/allegro/elasticsearch-analysis-morfologik

3. Define an ES analyzer for Polish texts containing:
- standard tokenizer
- synonym filter with alternative forms for months, e.g. kwiecień, kwi, IV.
- lowercase filter
- Morfologik-based lemmatizer
- lowercase filter (looks strange, but Morfologi produces capitalized base forms for proper names, so we have to  - lowercase them once more).
4. Define another analyzer for Polish, without the synonym filter.

In [ ]:
analysis_settings = {
    "settings": {
        "analysis": {
            "filter": {
                "synonym": {
                    # Definicja synonimów dla miesięcy
                    "type": "synonym",
                    "synonyms": [
                        "styczeń, sty, I",
                        "luty, lut, II",
                        "marzec, mar, III",
                        "kwiecień, kwi, IV",
                        "maj, V",
                        "czerwiec, cze, VI",
                        "lipiec, lip, VII",
                        "sierpień, sie, VIII",
                        "wrzesień, wrz, IX",
                        "październik, paź, X",
                        "listopad, lis, XI",
                        "grudzień, gru, XII"
                    ]
                }
            },
            "analyzer": {
                # Analizator z synonimami, małymi literami i lematyzacją
                "polish_analyzer_with_synonyms": {
                    "type": "custom",
                    "tokenizer": "standard",
                    "filter": ["synonym", "lowercase", "morfologik_stem", "lowercase"]
                },
                # Analizator bez synonimów
                "polish_analyzer_without_synonyms": {
                    "type": "custom",
                    "tokenizer": "standard",
                    "filter": ["lowercase", "morfologik_stem", "lowercase"]
                }
            }
        }
    },
    "mappings" : {
        "properties": {
            # Pole z analizą uwzględniającą synonimy
            "with_synonyms": {
                "type": "text",
                "analyzer": "polish_analyzer_with_synonyms"
            },
            # Pole z analizą bez synonimów
            "without_synonyms": {
                "type": "text",
                "analyzer": "polish_analyzer_without_synonyms"
            }
        }
    }
}

5. Define an ES index for storing the contents of the corpus FiQA-PL using both analyzers. Use different names for the fields analyzed with a different pipeline.

In [ ]:
from elasticsearch import Elasticsearch
from elasticsearch.helpers import bulk
import json
from datasets import load_dataset

es = Elasticsearch("http://localhost:9200")
print(es.info())

index_name = "fiqa-pl-index"
if es.indices.exists(index=index_name):
    es.indices.delete(index=index_name)
es.indices.create(index=index_name, body=analysis_settings, ignore=400)

c:\Users\gboch\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


{'name': 'node-1', 'cluster_name': 'my-application-cluster', 'cluster_uuid': 'MTibwZg5SaiWz0u8ARtMDw', 'version': {'number': '8.15.2', 'build_flavor': 'default', 'build_type': 'docker', 'build_hash': '98adf7bf6bb69b66ab95b761c9e5aadb0bb059a3', 'build_date': '2024-09-19T10:06:03.564235954Z', 'build_snapshot': False, 'lucene_version': '9.11.1', 'minimum_wire_compatibility_version': '7.17.0', 'minimum_index_compatibility_version': '7.0.0'}, 'tagline': 'You Know, for Search'}


C:\Users\gboch\AppData\Local\Temp\ipykernel_4460\3047365429.py:12: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  es.indices.create(index=index_name, body=analysis_settings, ignore=400)


ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'fiqa-pl-index'})

6. Load the data to the ES index.

In [ ]:
ds = load_dataset("clarin-knext/fiqa-pl", "corpus")
dataset = load_dataset("clarin-knext/fiqa-pl", 'corpus')
df = dataset['corpus'].to_pandas()
df

,_id,title,text
0,3,,"Nie mówię, że nie podoba mi się też pomysł szk..."
1,31,,Tak więc nic nie zapobiega fałszywym ocenom po...
2,56,,Nigdy nie możesz korzystać z FSA dla indywidua...
3,59,,Samsung stworzył LCD i inne technologie płaski...
4,63,,Oto wymagania SEC: Federalne przepisy dotycząc...
...,...,...,...
57633,599946,,">Cóż, po pierwsze, drogi to coś więcej niż hob..."
57634,599953,,"Tak, robią. Na dotacje dla firm farmaceutyczny..."
57635,599966,,">To bardzo smutne, że nie rozumiesz ludzkiej n..."
57636,599975,,„Czy Twój CTO pozwolił dużej grupie użyć „„adm...


In [ ]:
from elasticsearch.helpers import bulk, BulkIndexError

def load_data_to_es(index_name, data):
    """Function to load data into the index in Elasticsearch"""
    actions = [
        {
            "_index": index_name,
            "_id": doc["_id"],
            "_source": {
                "with_synonyms": doc["text"],
                "without_synonyms": doc["text"]
                }
        }
        for doc in data
    ]

    try:
        success, _ = bulk(es, actions)
        print(f"{success} documents loaded to index '{index_name}'")
    except BulkIndexError as e:
        print(f"{len(e.errors)} documents failed to load.")
        for error in e.errors:
            print(error)

index_name = 'fiqa-pl-index'
load_data_to_es(index_name, ds['corpus'])

57638 documents loaded to index 'fiqa-pl-index'


7. Determine the number of documents and the number of matches containing the word kwiecień (in any form) including and excluding the synonyms.

In [7]:
matches_with_synonyms = es.count(index=index_name, body={
    "query": {
        "match": {
            "with_synonyms": "kwiecień"  
        }
    }
})['count']
print(f"Matches, including synonyms: {matches_with_synonyms}")

matches_without_synonyms = es.count(index=index_name, body={
    "query": {
        "match": {
            "without_synonyms": "kwiecień"
        }
    }
})['count']
print(f"Matches, without synonyms: {matches_without_synonyms}")

Matches, including synonyms: 306
Matches, without synonyms: 257


8. Download the QA pairs for the FiQA-PL dataset.

In [ ]:
from datasets import load_dataset
import pandas as pd

queries_dataset = load_dataset('clarin-knext/fiqa-pl', 'queries')
corpus_dataset = load_dataset('clarin-knext/fiqa-pl', 'corpus')
qrels_dataset = load_dataset('clarin-knext/fiqa-pl-qrels', 'default')

queries = queries_dataset['queries'].to_pandas()
corpus = corpus_dataset['corpus'].to_pandas()
qrels = qrels_dataset['test'].to_pandas()

qrels['corpus-id'] = qrels['corpus-id'].astype(int)
corpus['_id'] = corpus['_id'].astype(int)
queries['_id'] = queries['_id'].astype(int)
qa_pairs = pd.merge(qrels, corpus, left_on="corpus-id", right_on="_id", how="left")
final_output = pd.merge(qa_pairs, queries, left_on="query-id", right_on="_id", suffixes=('_corpus', '_query'))

In [9]:
final_output

,query-id,corpus-id,score,_id_corpus,title_corpus,text_corpus,_id_query,title_query,text_query
0,8,566392,1,566392,,Poproś o ponowne wystawienie czeku właściwemu ...,8,,Jak zdeponować czek wystawiony na współpracown...
1,8,65404,1,65404,,Po prostu poproś współpracownika o podpisanie ...,8,,Jak zdeponować czek wystawiony na współpracown...
2,15,325273,1,325273,,Oczywiście że możesz. W sekcji Od przekazu pie...,15,,Czy mogę wysłać przekaz pieniężny z USPS jako ...
3,18,88124,1,88124,,Mylisz tutaj wiele rzeczy. Spółka B LLC będzie...,18,,1 EIN prowadzący działalność pod wieloma nazwa...
4,26,285255,1,285255,,"„Obawiam się, że wielkim mitem spółek z ograni...",26,,Ubieganie się o kredyt biznesowy i otrzymywani...
...,...,...,...,...,...,...,...,...,...
1701,11039,330058,1,330058,,"Zdecydowanie wkładałbym wystarczająco dużo, ab...",11039,,Spłać zadłużenie karty kredytowej lub zarób 40...
1702,11039,91183,1,91183,,"„Istnieje bardzo proste obliczenie, które odpo...",11039,,Spłać zadłużenie karty kredytowej lub zarób 40...
1703,11054,155053,1,155053,,„Nie ma specjalnej stawki dla krótkoterminowyc...,11054,,Podatek od krótkoterminowych zysków kapitałowy...
1704,11054,321015,1,321015,,„Najważniejsze jest to: w Stanach Zjednoczonyc...,11054,,Podatek od krótkoterminowych zysków kapitałowy...


9. Compute NDCG@5 for the QA dataset (the test subset) for the following setusp:
- synonyms enabled and disabled,
- lemmatization in the query enabled and disabled.

In [10]:
analysis_settings_no_lematization = {
    "settings": {
        "analysis": {
            "filter": {
                "synonym": {
                    "type": "synonym",
                    "synonyms": [
                        "styczeń, sty, I",
                        "luty, lut, II",
                        "marzec, mar, III",
                        "kwiecień, kwi, IV",
                        "maj, V",
                        "czerwiec, cze, VI",
                        "lipiec, lip, VII",
                        "sierpień, sie, VIII",
                        "wrzesień, wrz, IX",
                        "październik, paź, X",
                        "listopad, lis, XI",
                        "grudzień, gru, XII"
                    ]
                }
            },
            "analyzer": {
                "polish_analyzer_with_synonyms_no_lem": {
                    "type": "custom",
                    "tokenizer": "standard",
                    "filter": ["synonym", "lowercase", "lowercase"]
                },
                "polish_analyzer_without_synonyms_no_lem": {
                    "type": "custom",
                    "tokenizer": "standard",
                    "filter": ["lowercase", "lowercase"]
                }
            }
        }
    },
    "mappings" : {
        "properties": {
            "with_synonyms_no_lem": {
                "type": "text",
                "analyzer": "polish_analyzer_with_synonyms_no_lem"
            },
            "without_synonyms_no_lem": {
                "type": "text",
                "analyzer": "polish_analyzer_without_synonyms_no_lem"
            }
        }
    }
}

In [11]:
index_name_no_lem = "fiqa-pl-index_no_lem"
if es.indices.exists(index=index_name_no_lem):
    es.indices.delete(index=index_name_no_lem)
es.indices.create(index=index_name_no_lem, body=analysis_settings_no_lematization, ignore=400)

C:\Users\gboch\AppData\Local\Temp\ipykernel_4460\912292633.py:4: DeprecationWarning: Passing transport options in the API method is deprecated. Use 'Elasticsearch.options()' instead.
  es.indices.create(index=index_name_no_lem, body=analysis_settings_no_lematization, ignore=400)


ObjectApiResponse({'acknowledged': True, 'shards_acknowledged': True, 'index': 'fiqa-pl-index_no_lem'})

In [12]:
def load_data_to_es_no_lem(index_name_no_lem, data):
    actions = [
        {
            "_index": index_name_no_lem,
            "_id": doc["_id"],
            "_source": {
                "with_synonyms_no_lem": doc["text"],
                "without_synonyms_no_lem": doc["text"]
            }
        }
        for doc in data
    ]

    try:
        success, _ = bulk(es, actions)
        print(f"{success} documents loaded to index '{index_name_no_lem}'")
    except BulkIndexError as e:
        print(f"{len(e.errors)} documents failed to load.")
        for error in e.errors:
            print(error)


load_data_to_es_no_lem(index_name_no_lem, ds['corpus'])

57638 documents loaded to index 'fiqa-pl-index_no_lem'


In [ ]:
from datasets import load_dataset
import numpy as np

queries_dataset = load_dataset("clarin-knext/fiqa-pl", "queries")
queries = queries_dataset['queries']
queries = queries.to_pandas()

qa_dataset = load_dataset("clarin-knext/fiqa-pl-qrels")
test_data = qa_dataset['test']
qrels = test_data.to_pandas()


def calc_ndcg_k(true, predicted):
    """Function to calculate Normalized Discounted Cumulative Gain (NDCG) at rank k"""
    dcg = np.sum(predicted / np.log2(np.arange(2, len(true) + 2)))
    idcg = np.sum(true / np.log2(np.arange(2, len(true) + 2)))
    return dcg / idcg if idcg > 0 else 0


def count_ndcg(index, maps):
    """Function to count NDCG across queries"""
    max_matches = qrels.groupby('query-id')['corpus-id'].count().rename('count')
    
    ndcg_total = 0  
    num_queries = 0  

    for query_id in qrels['query-id'].unique():
        query = queries.loc[queries['_id'] == str(query_id), 'text'].iloc[0]

        query_blank = {
            "match": {
                maps: {
                    "query": query
                }
            }
        }
        
        resp = es.search(index=index, query=query_blank)
        corpus_ids = qrels.loc[qrels['query-id'] == query_id, 'corpus-id'].tolist()

        matches = min(max_matches.loc[query_id], 5)
        true = np.array([1 if i < matches else 0 for i in range(5)])
        
        predicted = np.zeros(5)

        for i, hit in enumerate(resp['hits']['hits'][:5]):
            predicted[i] = 1 if int(hit['_id']) in corpus_ids else 0
        ndcg_total += calc_ndcg_k(true, predicted)
        num_queries += 1

    return ndcg_total / num_queries if num_queries > 0 else 0

In [17]:
mean_ndcg = count_ndcg("fiqa-pl-index", "with_synonyms")
print(f"The average NDCG is {mean_ndcg} for the index with synonyms and lemmatization.")

The average NDCG is 0.18512911307977398 for the index with synonyms and lemmatization.


In [18]:
mean_ndcg = count_ndcg("fiqa-pl-index", "without_synonyms")
print(f"The average NDCG is {mean_ndcg} for the index without synonyms and lemmatization.")

The average NDCG is 0.18512911307977398 for the index without synonyms and lemmatization.


In [19]:
mean_ndcg = count_ndcg("fiqa-pl-index_no_lem", "with_synonyms_no_lem")
print(f"The average NDCG is {mean_ndcg} for the index with synonyms and no lemmatization.")

The average NDCG is 0.13854570378524383 for the index with synonyms and no lemmatization.


In [20]:
mean_ndcg = count_ndcg("fiqa-pl-index_no_lem", "without_synonyms_no_lem")
print(f"The average NDCG is {mean_ndcg} for the index with synonyms and lemmatization.")

The average NDCG is 0.13854570378524383 for the index with synonyms and lemmatization.


In [15]:
# indices = es.cat.indices(format="json")
# index_names = [index['index'] for index in indices]
# print(index_names)

# for index in index_names:
#     es.indices.delete(index=index)
#     print(f"Usunięto indeks: {index}")

## Questions

1. What are the strengths and weaknesses of regular expressions versus full text search regarding processing of text?

Wyrażenia regularne:
- Umożliwiają bardzo precyzyjne dopasowanie określonych wzorców w tekście
- Zapewniają precyzyjną kontrolę nad wyszukiwaniem, np. określonych słów w określonych kontekstach.
- Są dobre w rozpoznawaniu niekompletnych lub niejednoznacznych wzorców, ale są ograniczone do tego, co programuje użytkownik.
- Działają na poziomie znaków, nie biorąc pod uwagę znaczenia i kontekstu słów.
- Łatwiejsze w użyciu

Wyszukiwanie pełnotekstowe (FTS):
- Zoptymalizowane do szybkiego wyszukiwania dużych zestawów danych (np. ElasticSearch).
- Obsługuje lematyzację, stemming i synonimy, co zwiększa dokładność wyszukiwań opartych na znaczeniu.
- Mniej precyzyjne niż wyrażenia regularne, ponieważ działa na indeksie słów (nie zawsze: wszystko zależy od tego, jakiego wyrażenia regularnego użył użytkownik)
- Wymaga bardziej złożonej konfiguracji i ponownego indeksowania po zmianach danych.
- Wyniki nie są tak rozbieżne, nie zależą od tego, co użytkownik wpisuje/myśli

2. Can an LLM be applied in the context of searching for documents? Justify your answer, excluding the obvious observation that an LLM can be used to formulate the answer.

Tak, modele językowe (LLM) mogą obsługiwać wyszukiwanie dokumentów w zaawansowanych scenariuszach:

- LLM rozumieją semantykę i intencję zapytań, wykraczając poza prostą analizę słów kluczowych.
- Mogą generować podsumowania dokumentów i kluczowe informacje, ułatwiając przeglądanie wyników.
- Rozpoznają warianty językowe, synonimy i różne style.
- Uzupełniają zapytania i lepiej dopasowują dokumenty, nawet jeśli zapytanie jest nieprecyzyjne.
- Umożliwiają personalizację wyników na podstawie preferencji użytkownika.
- Mogą rekomendować dokumenty powiązane z poprzednimi wyszukiwaniami.